# Tools

## Github

#### Clone a repository based on url

In [45]:
from typing import Optional, Dict, Any
from git import Repo, GitCommandError
from langchain.tools import tool
import os, shutil


@tool("git_clone")
def git_clone_tool(
    repo_url: str,
    dest: str,
    branch: Optional[str] = None,
    overwrite: bool = False,
) -> Dict[str, Any]:
    """
    Clone a Git repository into ./repositories/{dest} using GitPython.

    Args:
        repo_url: HTTPS or SSH URL of the repository.
        dest: Name of the destination folder for the clone inside ./repositories/.
        branch: Optional branch to check out.
        overwrite: If True, overwrite existing destination folder.
    Returns:
        A dict with success (bool), dest (str), and error/stdout messages.
    """

    try:
        # Ensure repositories/ root exists
        root_dir = os.path.join(os.getcwd(), "repositories")
        os.makedirs(root_dir, exist_ok=True)

        # Full destination path inside repositories/
        full_dest = os.path.join(root_dir, dest)

        # Handle overwrite
        if os.path.exists(full_dest):
            if overwrite:
                shutil.rmtree(full_dest)
            else:
                return {"success": False, "error": f"Destination {full_dest} already exists."}


        # Clone options
        kwargs = {}
        if branch:
            kwargs["branch"] = branch
            full_dest = f"{full_dest}/{branch}"

        os.makedirs(full_dest, exist_ok=True)
        repo = Repo.clone_from(repo_url, full_dest, **kwargs)

        return {
            "success": True,
            "dest": full_dest,
            "branch": repo.active_branch.name if not repo.head.is_detached else "detached",
            "error": None,
        }
    except GitCommandError as e:
        return {"success": False, "dest": dest, "error": str(e)}
    except Exception as e:
        return {"success": False, "dest": dest, "error": str(e)}

#### Add Github MCP server based on url

In [46]:
from typing import Dict, Any
from langchain.tools import tool
from experiments.utils.mcp_client_factory import load_mcp_config, save_mcp_config, create_mcp_client_from_config


@tool("add_mcp_server")
def add_mcp_server_tool(
    name: str,
    url: str,
    transport: str = "streamable_http",
) -> Dict[str, Any]:
    """
    Add a new MCP server to the configuration.
    
    Args:
        name: Name identifier for the MCP server
        url: URL of the MCP server
        transport: Transport type (default: "streamable_http")
    Returns:
        Dict with success status and current config

    Note: 
        MCP tools accessed via clients are not hot reloaded or dynamically  updated,
        a new agent or tool instance has to be  
    """
    try:
        # Load existing config
        config = load_mcp_config()
        
        # Add new server
        config[name] = {
            "url": url,
            "transport": transport
        }
    
        save_mcp_config(config)
        
        return {
            "success": True,
            "message": f"Added MCP server '{name}'",
            "config": config
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "config": {}
        }
    
@tool("add_github_repository_as_mcp_server")
def add_github_repository_as_mcp_tool(repo_url: str, server_name: str) -> Dict[str, Any]:
    """
    Add a GitHub repository as an MCP server using gitmcp.io.
    
    Args:
        repo_url: GitHub repository URL (e.g., https://github.com/owner/repo)
        server_name: Name identifier for the MCP server
    Returns:
        Dict with success status and current config
    """
    # Extract the repository path from the GitHub URL
    if "github.com/" in repo_url:
        # Extract everything after github.com/
        repo_path = repo_url.split("github.com/", 1)[1]
        # Remove .git suffix if present
        if repo_path.endswith(".git"):
            repo_path = repo_path[:-4]
        gitmcp_url = f"https://gitmcp.io/{repo_path}"
    else:
        raise ValueError("Invalid GitHub repository URL")
    
    tool_input = {
        "name": server_name,
        "url": gitmcp_url,
    }

    return add_mcp_server_tool.invoke(tool_input)

    

@tool("remove_mcp_server")
def remove_mcp_server_tool(
    name: str,
) -> Dict[str, Any]:
    """
    Remove an MCP server from the configuration.
    
    Args:
        name: Name identifier of the MCP server to remove
    Returns:
        Dict with success status and current config

    Note: 
        MCP tools accessed via clients are not hot reloaded or dynamically  updated,
        a new agent or tool instance has to be 
    """
    try:
        config = load_mcp_config()
        
        if name not in config:
            return {
                "success": False,
                "error": f"MCP server '{name}' not found",
                "config": config
            }
        
        del config[name]
        save_mcp_config(config)
        
        return {
            "success": True,
            "message": f"Removed MCP server '{name}'",
            "config": config
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "config": {}
        }

@tool("list_mcp_servers")
def list_mcp_servers() -> Dict[str, Any]:
    """
    List all configured MCP servers.
    
    Returns:
        Dict with success status and list of servers
    """
    try:
        config = load_mcp_config()
        return {
            "success": True,
            "servers": list(config.keys()),
            "config": config
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "config": {}
        }

In [ ]:
from langchain_core.messages import HumanMessage

import sys
import os
# Add the parent directory to sys.path so 'experiments' can be imported
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
from experiments.utils.agent_factory import create_agent_with_valid_tools

tools = [add_github_repository_as_mcp_tool, add_mcp_server_tool, remove_mcp_server_tool, list_mcp_servers, git_clone_tool]

agent = create_agent_with_valid_tools(
    "openai:gpt-5-nano",
    tools=tools,
    prompt="""Act as an assistant.
                When using tools:
                - Use tools if relevant before answering.
            """
)


stream = agent.astream({"messages": [HumanMessage("Can you clone the github repository: https://github.com/simonskodt/arch-reconstruct-ai")]})
async for chunk in stream:
    print(chunk)


result = await agent.ainvoke({"messages": [HumanMessage("Can you list the MCP servers available")]})
print(result)

result = await agent.ainvoke({"messages": [HumanMessage("Can you take the following github repository: https://github.com/simonskodt/arch-reconstruct-ai, and make it into a MCP server")]})
print(result)




client = create_mcp_client_from_config()
mcp_tools = await client.get_tools() 
tools += mcp_tools

[print(tool.name) for tool in tools]

agent = create_agent_with_valid_tools(
    "openai:gpt-5-nano",
    tools=tools, # Tools cannot be dynamically  or hot reloaded?, agent has to be recreated  
    prompt="""Act as an assistant.
                When using tools:
                - Use tools if relevant before answering.
            """
)
result = await agent.ainvoke({"messages": [HumanMessage("Can you list the MCP servers available")]})
print(result)



{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iqNmvENZzhZ7Wh3kntQPXqkJ', 'function': {'arguments': '{"repo_url":"https://github.com/simonskodt/arch-reconstruct-ai","dest":"arch-reconstruct-ai","branch":null,"overwrite":false}', 'name': 'git_clone'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 308, 'prompt_tokens': 511, 'total_tokens': 819, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CLo4ayGmdBLSAkgAUugCxyrMtnlJ7', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--277e2183-d669-49fd-844c-d2a1f3f364b6-0', tool_calls=[{'name': 'git_clone', 'args': {'repo_url': 'https://github.com/simonskodt/arch-reconstruct-ai', '

In [ ]:
from typing import Optional, Dict, Any
from git import Repo, GitCommandError
from langchain.tools import tool
import os, shutil

from gitingest import ingest, ingest_async

@tool("extract_repository_details")
async def extract_repository_details(
    local_repository_path: Optional[str],
    github_url: Optional[str] = None,
    output_path: Optional[str] = "-",
) -> Dict[str, Any]:
    """
    Extract and ingest a Git repository (local or remote) into a readable LLM format.

    Args:
        local_repository_path: Path to a local repository directory.
        github_url: HTTPS URL of a remote GitHub repository.
        output_path: Output path for the extraction (default: "-" for stdout).
    Returns:
        A dict with summary (str), tree (str), and content (str) of the repository.
    """

    try:        
        exclude_patterns = {
            "*.pyc",
            "__pycache__",
            ".git",
            ".venv",
            "venv",
            "env",
            "node_modules",
            ".DS_Store",
            "*.log",
            ".pytest_cache",
            "*.egg-info",
            "dist",
            "build",
            "*.lock",
            ".pylintrc"
        }
        if local_repository_path:
            summary, tree, content = await ingest_async(local_repository_path, exclude_patterns=exclude_patterns, output=output_path)
        elif github_url:
            summary, tree, content = await ingest_async(github_url, exclude_patterns=exclude_patterns, output=output_path)
        else:
            return {"success": False, "error": "Either local_repository_path or github_url must be provided"}

        extraction = {"summary": summary, "tree": tree, "content": content} 

        return extraction
    except Exception as e:
        return {"success": False, "error": str(e)}

2025-10-08 01:17:44.479 | INFO     | logging:callHandlers:1737 | Exception in execute request:
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[48], line 1
----> 1 agent = create_agent(
      2     "openai:gpt-5-nano",
      3     tools=[list_current_directory, change_directory],
      4     prompt=f"Act as an assistant, that can navigate the file system using the tools provided.",
      5 )
      7 result = agent.invoke({"messages": [HumanMessage("Can you list the current directory?")]})
      8 print(result)

NameError: name 'create_agent' is not defined
2025-10-08 01:18:07.434 | INFO     | logging:callHandlers:1737 | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-08 01:18:10.335 | INFO     | logging:callHandlers:1737 | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-08 01:40:54.139 | INFO     |

In [3]:
tool_input = {
    "local_repository_path": "./repositories/api",
    "output_path": None,
    "github_url": None,
}

print(extract_repository_details.name)

output = await extract_repository_details.ainvoke(tool_input)

for key in output:
    if key != "content":  
        print(f"{key}:  {output[key]}")

extract_repository_details
summary:  Directory: ./repositories/api
Files analyzed: 608

Estimated tokens: 3.7M
tree:  Directory structure:
└── api/
    ├── archlens.json
    ├── cleanup.sh
    ├── default.env
    ├── default.fmd.cfg
    ├── default_api.cfg
    ├── default_docker.cfg
    ├── default_docker_v8.cfg
    ├── docker-compose.yml
    ├── Dockerfile
    ├── Dockerfile.development
    ├── env_var_defs_default.py
    ├── generate_configs.sh
    ├── install_stanza_models.py
    ├── LICENSE
    ├── MANIFEST.in
    ├── requirements.txt
    ├── run_tests.sh
    ├── setup.py
    ├── start.py
    ├── truckconfig.json
    ├── zeeguu_api.wsgi
    ├── .envrc
    ├── tools/
    │   ├── __init__.py
    │   ├── activity_monitor.py
    │   ├── add_captions_from_file.py
    │   ├── add_feed.py
    │   ├── add_images_to_articles.py
    │   ├── add_videos_and_captions_from_file.py
    │   ├── analyze_classroom_recommendations.py
    │   ├── anonymize_users.py
    │   ├── article_crawler.py
    │

## ArchLens

In [20]:
from typing import Optional, Dict, Any, List
from logging import Handler
from pydantic import BaseModel


class ArchLensConfig(BaseModel):
  name: str
  rootFolder: str
  views: Dict[str, Dict[str, List[Dict[str, Any]]]]
  saveLocation:str  = "./diagrams/"

#create Archlensobject example
viewsJson = {"top-level-view-depth-1": {
      "packages": [
        {
          "path": "*",
          "depth": 1
        }
      ]
    },
    "top-level-view-depth-2": {
      "packages": [
        {
          "path": "*",
          "depth": 2
        }
      ]
    }}

archlensObject = ArchLensConfig(name="tets", rootFolder='zeeguu', views=viewsJson)

print(archlensObject)



name='tets' rootFolder='zeeguu' views={'top-level-view-depth-1': {'packages': [{'path': '*', 'depth': 1}]}, 'top-level-view-depth-2': {'packages': [{'path': '*', 'depth': 2}]}} saveLocation='./diagrams/'


In [57]:
import json
from pathlib import Path

@tool('init_archLens')
def init_archLens(repo_url: str):
    """"If you have cloned the arch-reconstruct-ai repository, initialize archLens."""

    repo_name = repo_url.rstrip('/').split('/')[-1]
    if repo_name.endswith('.git'):
        repo_name = repo_name[:-4]
    print(f"Repository name: {repo_name}")

    repo_path = "/Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/"+"api"
    os.chdir(repo_path)

    if os.path.exists("archlens.json"):
        print("archlens.json already exists, skipping initialization.")
    else:
        os.system("archlens init")

    return "Initialized archLens"

@tool('run_archLens')
def run_archLens(repo_url: str):
    """"Run archLens on the cloned repository."""

    repo_name = repo_url.rstrip('/').split('/')[-1]
    if repo_name.endswith('.git'):
        repo_name = repo_name[:-4]
    print(f"Repository name: {repo_name}")
    
    repo_path = "/Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/"+"api"
    os.chdir(repo_path)
      
    if not os.path.exists("archlens.json"):
        return "archlens.json does not exist. Please run init_archLens first."

    os.system(f"archlens render")

    return "Ran archLens"


@tool('read_archLens_config_file')
def read_archLens_config_file(repo_url: str):
    """"Reads the content of the archlens.json file."""

    repo_name = repo_url.rstrip('/').split('/')[-1]
    if repo_name.endswith('.git'):
        repo_name = repo_name[:-4]
    print(f"Repository name: {repo_name}")

    repo_path = "/Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/"+"api"
    os.chdir(repo_path)

    config_path = "archlens.json"
    if not os.path.exists(config_path):
        return "archlens.json does not exist. Please run init_archLens first."
    
    with open('archlens.json', 'r') as file:
        data = json.load(file)

    print(json.dumps(data, indent=4))
    return "Read config file"


@tool('write_archLens_config_file')
def write_archLens_config_file(repo_url: str, arch: ArchLensConfig):
    """"Writes content to the archlens.json file.
        args: 
    """
    repo_name = repo_url.rstrip('/').split('/')[-1]
    if repo_name.endswith('.git'):
        repo_name = repo_name[:-4]
    print(f"Repository name: {repo_name}")

    repo_path = "/Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/"+"api"
    os.chdir(repo_path)

    config_path = "archlens.json"
    with open(config_path, 'w') as file:
        json.dump(arch.__dict__, file)
    
    return "Wrote to config file"

@tool('create_ArchLensConfig_Object')
def create_ArchLensConfig_Object(packageName:str, path: str, depth: int) -> ArchLensConfig:
    """"Creates an ArchLensConfig object, which is used when writing to the archlens.json file. This is the structure of the ArchLensConfig object:
    viewsJson = {"top-level-view-depth-1": {
      "packages": [
        {
          "path": "*",
          "depth": 1
        }
      ]
    },
    "top-level-view-depth-2": {
      "packages": [
        {
          "path": "*",
          "depth": 2
        }
      ]
    }}
    """

    viewsJson = {packageName: {
      "packages": [
        {
          "path": path,
          "depth": depth
        }
      ]
    }
    }

    archlensObject = ArchLensConfig(name='testing' ,rootFolder='zeeguu', views=viewsJson)
    return archlensObject

@tool('add_view_to_ArchLensConfig_Object')
def add_view_to_ArchLensConfig_Object(archlensObject: ArchLensConfig, packageName:str, path:str, depth: int) -> ArchLensConfig:

  """"Adds a view to an existing ArchLensConfig object. 
          args: 
              archlensObject: The existing ArchLensConfig object.
              name: The name of the view to add.
  """

  archlensObject.views[packageName] = {
      "packages": [
        {
          "path": path,
          "depth": depth
        }
      ]
    }
  return archlensObject


archConfig = create_ArchLensConfig_Object({"packageName": "top-level-view-depth-1", "path": "*", "depth": 1})
ar = add_view_to_ArchLensConfig_Object({"archlensObject": archConfig, "packageName": "top-level-view-depth-2", "path": "*", "depth": 2})
write_archLens_config_file({"repo_url": "https://github.com/zeeguu/api.git", "arch": ar})

run_archLens('https://github.com/zeeguu/api.git')


NameError: name 'ArchLensConfig' is not defined

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    "openai:gpt-5-nano",
    tools=[init_archLens, run_archLens, read_archLens_config_file, write_archLens_config_file, create_ArchLensConfig_Object],
    prompt=f"Act as an assistant, that does architectural reconstruction of software repositories. Archlens is a tool which can be used for python repositories",
)

#result = agent.invoke({"messages": [HumanMessage("Can you clone the following github repository: https://github.com/simonskodt/arch-reconstruct-ai, feel free to overwrite if a clone already exists, ")]})
result = agent.invoke({"messages": [HumanMessage("I want you to create an ArchLensConfig object with the name agent, depth 3 and path *. Afterwards write the generated ArchlensConfig object to the archlens.json file and run archlens render. The repository url is https://github.com/zeeguu/api.git")]})
print(result)


Repository name: api
Repository name: api
analyzing /Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/api/zeeguu
analyzing /Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/api/zeeguu/core
analyzing /Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/api/zeeguu/core/user_activity_hooks
analyzing /Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/api/zeeguu/core/account_management
analyzing /Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/api/zeeguu/core/word_filter
analyzing /Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/api/zeeguu/core/word_scheduling
analyzing /Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-recons

## Navigation tools

### List current directory

In [44]:
import os

@tool("List_current_directory")
def list_current_directory() -> list[str]:
    """List the contents of the current directory."""
    return os.listdir(".")

### Change directory

In [58]:
@tool("Change_directory")
def change_directory(path: str) -> None:
    """Change the current working directory."""
    os.chdir(path)

### Current working directory

In [59]:
@tool("current_working_directory")
def current_working_directory() -> str:
    """Return the current working directory."""
    return os.getcwd()

### Get parent and child directory

In [54]:
from typing import Any 
@tool("get_parent_and_child_directory")
def get_parent_and_child_directory(ls: bool = True) -> dict[str, Any]:
    """Return the parent and child directory of the current working directory."""
    current_dir = os.getcwd()
    parent_dir = os.path.dirname(current_dir)
    child_dir = os.path.join(current_dir, "child")
    dict_result: dict[str, Any] = {"parent": parent_dir, "child": child_dir}
    if ls:
        dict_result["parent_contents"] = os.listdir(parent_dir)
        dict_result["child_contents"] = os.listdir(child_dir) if os.path.exists(child_dir) else []

    return dict_result

In [ ]:
@tool('run_archLens')
def run_archLens(repo_url: str):
    """"Run archLens on the cloned repository."""

    repo_name = repo_url.rstrip('/').split('/')[-1]
    if repo_name.endswith('.git'):
        repo_name = repo_name[:-4]
    print(f"Repository name: {repo_name}")
    
    repo_path = "/Users/nikolajworsoelarsen/Desktop/CSKandidat/Thesis/arch-reconstruct-ai/experiments/tool_calling/repositories/"+"api"
    os.chdir(repo_path)
      
    if not os.path.exists("archlens.json"):
        return "archlens.json does not exist. Please run init_archLens first."

    os.system(f"archlens render")

    return "Ran archLens"


### Navigation example

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage


## clone repository for testing

tool_input = {
    "repo_url": "https://github.com/zeeguu/api",
    "dest": "zeeguu-api",
}

if False:
    output = git_clone_tool.invoke(tool_input)
    print(output)


## output repository details
tool_input = {
    "local_repository_path": "./repositories/zeeguu-api",
    "output_path": "./repositories/zeeguu-api/repo_extraction.txt",
}

if False:
    output = await extract_repository_details.ainvoke(tool_input)

    print(output["summary"])
    print(output["tree"])

queries = []

tools = [list_current_directory, change_directory, current_working_directory, get_parent_and_child_directory, extract_repository_details]
tool_descriptions = [tool.description for tool in tools]
agent = create_agent(
    "openai:gpt-5-nano",
    tools=tools,
    prompt=f"Act as an assistant, that can navigate the file system using the tools provided.\
    The available tools are: {tool_descriptions}"
)

repo_to_clone = "https://github.com/zeeguu/api"
clone = agent.invoke(
    f"Can you clone the following github repository: {repo_to_clone}, feel free to overwrite if a clone already exists, "
)
repo_extraction = agent.invoke(
    f"Can you extract the repository details of the cloned repository in, and save the output to the root folder of zeeguu-api/repo_extraction.txt"
)

cwd = agent.invoke({"messages": [HumanMessage("Can you list the current directory?")]})
tree = agent.invoke({"messages": [HumanMessage("Can you get the directory tree from the repository?")]})
chdir = agent.invoke({"messages": [HumanMessage("Can you navigate to the audio-engleza folder?")]})
info = agent.invoke({"messages": [HumanMessage("Can you show me the parent and child directories?")]})
archlens_init = agent.invoke({"messages": [HumanMessage("Can you initialize archLens in the cloned repository?")]})
archlens_render = agent.invoke({"messages": [HumanMessage("Can you render an archLens view of the repository?")]})

queries += [clone, repo_extraction, cwd, tree, chdir, info, archlens_init, archlens_render]

for query in queries:
    for message in query["message"]:
        message.pretty_print()


{'success': False, 'error': 'Destination /Users/thomas/Desktop/arch-reconstruct-ai/experiments/tool_calling/repositories/zeeguu-api already exists.'}
Directory: ./repositories/zeeguu-api
Files analyzed: 618

Directory structure:
└── zeeguu-api/
    ├── archlens.json
    ├── cleanup.sh
    ├── default.env
    ├── default.fmd.cfg
    ├── default_api.cfg
    ├── default_docker.cfg
    ├── default_docker_v8.cfg
    ├── docker-compose.yml
    ├── Dockerfile
    ├── Dockerfile.development
    ├── env_var_defs_default.py
    ├── generate_configs.sh
    ├── install_stanza_models.py
    ├── LICENSE
    ├── MANIFEST.in
    ├── requirements.txt
    ├── run_tests.sh
    ├── setup.py
    ├── start.py
    ├── truckconfig.json
    ├── zeeguu_api.wsgi
    ├── .envrc
    ├── tools/
    │   ├── __init__.py
    │   ├── activity_monitor.py
    │   ├── add_captions_from_file.py
    │   ├── add_feed.py
    │   ├── add_images_to_articles.py
    │   ├── add_videos_and_captions_from_file.py
    │   ├── analyze

In [ ]:
@tool('run_archlens')
def run_archlens(path: str):
    """"Run archLens from the <path>.
    
    args:
    - path: The path to the repository to run archLens on.


    """
    change_directory(path)
      
    if not os.path.exists("archlens.json"):
        return "archlens.json does not exist. Please run init_archLens first."

    exit_code = os.system(f"archlens render")

    return {"message": "Ran archLens", "exit_code": exit_code}



## Create navigation class